In [1]:
import torch
import torch.nn.functional as F
from poisoned_dataset import PoisonedDataset
from datasets import get_dataset
import numpy as np
from spikingjelly.datasets.n_mnist import NMNIST
import copy
import os
import csv

In [2]:
def ssim(img1, img2, C1=0.01**2, C2=0.03**2):
    """
    Compute SSIM (Structural Similarity Index) between two images.
    :param img1: First image (torch tensor)
    :param img2: Second image (torch tensor)
    :param C1: Stabilization constant 1
    :param C2: Stabilization constant 2
    :return: SSIM value
    """
    mu1 = F.avg_pool2d(img1, 3, 1, 0)
    mu2 = F.avg_pool2d(img2, 3, 1, 0)
    
    sigma1 = F.avg_pool2d(img1 * img1, 3, 1, 0) - mu1 * mu1
    sigma2 = F.avg_pool2d(img2 * img2, 3, 1, 0) - mu2 * mu2
    sigma12 = F.avg_pool2d(img1 * img2, 3, 1, 0) - mu1 * mu2

    ssim_map = ((2 * mu1 * mu2 + C1) * (2 * sigma12 + C2)) / ((mu1 * mu1 + mu2 * mu2 + C1) * (sigma1 + sigma2 + C2))
    return ssim_map.mean()

def calculate_average_ssim(data_clean, data_backdoored):
    """
    Calculate the average SSIM over all samples, frames, and polarities.
    :param data_clean: Clean data (numpy array of shape [N, T, C, H, W])
    :param data_backdoored: Backdoored data (numpy array of shape [N, T, C, H, W])
    :return: Average SSIM value
    """
    if not torch.is_tensor(data_clean):
        data_clean = torch.tensor(data_clean, dtype=torch.float32)
    if not torch.is_tensor(data_backdoored):
        data_backdoored = torch.tensor(data_backdoored, dtype=torch.float32)

    N, T, C, H, W = data_clean.shape
    total_ssim = 0.0
    count = 0
    
    for i in range(N):
        for t in range(T):
            for c in range(C):
                img_clean = data_clean[i, t, c].unsqueeze(0).unsqueeze(0)
                img_backdoored = data_backdoored[i, t, c].unsqueeze(0).unsqueeze(0)
                total_ssim += ssim(img_clean, img_backdoored)
                count += 1
                
    average_ssim = total_ssim / count
    return average_ssim

In [12]:
def save_result(dataset, epsilon,pos, polarity, trigger_size , trigger_type,
                            frame_gap, ssim):
    save_path = "experiments"
    # Create a folder for the experiments, by default named 'experiments'
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Create if not exists a csv file, appending the new info
    path = '{}/ssim_results.csv'.format(save_path)
    header = ['dataset', 'epsilon', 'pos','polarity', 'trigger_size','type', 'frame_gaps', "ssim"]

    if not os.path.exists(path):
        with open(path, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(header)

    # Append the new info to the csv file
    with open(path, 'a') as f:
        writer = csv.writer(f)
        writer.writerow([dataset, epsilon,pos, polarity, trigger_size , trigger_type,
                            frame_gap, ssim])

In [13]:
train_set = NMNIST("data/mnist", train=True, data_type='frame',
                           split_by='number', frames_number=16)

clean_data = np.array([np.array(x[0])for x in train_set])
# print(clean_data.shape)

# clean_train_data = PoisonedDataset(train_set,0, mode='train', epsilon=0,
#                                  pos="top-left", attack_type="static", time_step=16,
#                                  trigger_size=0.1, dataname="mnist",
#                                  polarity=1, n_masks=2, least=True, most_polarity=True, frame_gap = 1)
# poisoned_train_data = PoisonedDataset(train_set,0, mode='train', epsilon=1,
#                                  pos="top-left", attack_type="static", time_step=16,
#                                  trigger_size=0.1, dataname="mnist",
#                                  polarity=1, n_masks=2, least=True, most_polarity=True, frame_gap = 1)

The directory [data/mnist\frames_number_16_split_by_number] already exists.


In [5]:
exp1_types = ["static", "rs"]
epsilons = []
dataset="mnist"
epsilon=1
pos="top-left"
polarity=1
trigger_size=0.1
frame_gap = 1
for trigger_type in exp1_types:
    poisoned_train_data = PoisonedDataset(train_set,0, mode='train', epsilon=1,
                                 pos="top-left", attack_type=trigger_type, time_step=16,
                                 trigger_size=0.1, dataname="mnist",
                                 polarity=1, n_masks=2, least=True, most_polarity=True, frame_gap = 1)
    average_ssim_value = calculate_average_ssim(clean_data, poisoned_train_data.data)
    average_ssim_value = float(average_ssim_value)
    print("Average SSIM:", average_ssim_value) 
    save_result(dataset, epsilon,pos, polarity, trigger_size , trigger_type,
                            frame_gap, average_ssim_value)


[!] Generating train Bad Imgs
Save frames to [backdoor_static.gif].
Injecting Over: Bad Imgs: 10000. Clean Imgs: 0. Epsilon: 1
Average SSIM: 0.9952746033668518
[!] Generating train Bad Imgs
Save frames to [backdoor_rs.gif].
Injecting Over: Bad Imgs: 10000. Clean Imgs: 0. Epsilon: 1
Average SSIM: 0.9939156174659729


In [14]:
trigger_type = "static"
dataset="mnist"
epsilon=1
pos="top-left"
polarity=3
trigger_size=0.01
frame_gap = 2
poisoned_train_data = PoisonedDataset(train_set,0, mode='train', epsilon=1,
                                pos=pos, attack_type=trigger_type, time_step=16,
                                trigger_size=trigger_size, dataname=dataset,
                                polarity=polarity, n_masks=2, least=True, most_polarity=True, frame_gap = frame_gap)
average_ssim_value = calculate_average_ssim(clean_data, poisoned_train_data.data)
average_ssim_value = float(average_ssim_value)
print("Average SSIM:", average_ssim_value) 
save_result(dataset, epsilon,pos, polarity, trigger_size , trigger_type,
                        frame_gap, average_ssim_value)

[!] Generating train Bad Imgs
Save frames to [backdoor_static.gif].
Injecting Over: Bad Imgs: 60000. Clean Imgs: 0. Epsilon: 1
Average SSIM: 0.9999917149543762


In [17]:
from spikingjelly.datasets import play_frame

            
play_frame(clean_data[11121],"c.gif")
play_frame(poisoned_train_data.data[11121], "p.gif")


Save frames to [c.gif].
Save frames to [p.gif].


In [7]:
# print("Average SSIM:", float(average_ssim_value))